In [3]:
pip install numpy pandas matplotlib seaborn tensorflow opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   -- ------------------------------------- 2.6/38.8 MB 13.7 MB/s eta 0:00:03
   -- ------------------------------------- 2.6/38.8 MB 13.7 MB/s eta 0:00:03
   ----- ---------------------------------- 5.0/38.8 MB 8.2 MB/s eta 0:00:05
   ------ --------------------------------- 6.0/38.8 MB 7.4 MB/s eta 0:00:05
   ------- -------------------------------- 6.8/38.8 MB 6.7 MB/s eta 0:00:05
   ------- -------------------------------- 7.3/38.8 MB 5.9 MB/s eta 0:00:06
   -------- ------------------------------- 7.9/38.8 MB 5.6 MB/s eta 0:00:06
   -------- ------------------------------- 8.7/38.8 MB 5.4 MB/s eta 0:00:06
   --------- ------------------------------ 9.7/38.8 MB 5.3 MB/s eta 0:00:06
   ----------- ---------------------------- 11.0/38.8 MB 5.3 MB/s eta 0:00:06
   ------------ --------------------------- 12.3/38.8 MB 5.4 MB/s eta 0:00:05
   ------------- -------------------------- 13.1/38.8 MB 5.3 MB/s eta 0:00:05
 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.regularizers import l1, l2

In [47]:
# Load the dataset
sign_minist_train = pd.read_csv(r"D:\MCA project\Extra Projects\Sign\sign_mnist_train\sign_mnist_train.csv")
sign_minist_test = pd.read_csv(r"D:\MCA project\Extra Projects\Sign\sign_mnist_test\sign_mnist_test.csv")

X_train_sign = sign_minist_train.drop('label', axis=1).values.reshape(-1, 28, 28, 1)
y_train_sign = to_categorical(sign_minist_train['label'] - 1, num_classes=24)

X_test_sign = sign_minist_test.drop('label', axis=1).values.reshape(-1, 28, 28, 1)
y_test_sign = to_categorical(sign_minist_test['label'] - 1, num_classes=24)

# Normalize the dataset
X_train_sign = X_train_sign / 255.0
X_test_sign = X_test_sign / 255.0

# Check the shape of the datasets
print(f"Training dataset shape: {X_train_sign.shape}, Labels shape: {y_train_sign.shape}")
print(f"Testing dataset shape: {X_test_sign.shape}, Labels shape: {y_test_sign.shape}")

# Augment Sign Language data
datagen_sign = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen_sign = ImageDataGenerator(rescale=1./255)

datagen_sign.fit(X_train_sign)
# datagen_sign.flow()

print("Data Augmentation Applied to Sign Language Dataset")



Training dataset shape: (27455, 28, 28, 1), Labels shape: (27455, 24)
Testing dataset shape: (7172, 28, 28, 1), Labels shape: (7172, 24)
Data Augmentation Applied to Sign Language Dataset


In [6]:
# Load emotion dataset
trainingset = np.loadtxt(r"D:\MCA project\Extra Projects\fer2013_training_onehot.csv", delimiter=',')
testingset = np.loadtxt(r"D:\MCA project\Extra Projects\fer2013_publictest_onehot.csv", delimiter=',')

# Extract pixels and labels
pixels = trainingset['pixels'].apply(lambda pixel_sequence: np.array(pixel_sequence.split(), dtype=np.uint8))
labels = trainingset['emotion']


# Split the data into input and output
x_train_fer = trainingset[:, :-7]
y_train_fer = trainingset[:, -7:]
x_test_fer = testingset[:, :-7]
y_test_fer = testingset[:, -7:]

# Reshape the input data
x_train_fer = x_train_fer.reshape(-1, 48, 48, 1)
x_test_fer = x_test_fer.reshape(-1, 48, 48, 1)

# Normalize the dataset
x_train_fer = x_train_fer / 255.0
x_test_fer = x_test_fer / 255.0

print("Emotion Detection Dataset Loaded and Preprocessed")
print(f"x_train_fer shape: {x_train_fer.shape}, y_train_fer shape: {y_train_fer.shape}")
print(f"x_test_fer shape: {x_test_fer.shape}, y_test_fer shape: {y_test_fer.shape}")

Emotion Detection Dataset Loaded and Preprocessed
x_train_fer shape: (28709, 48, 48, 1), y_train_fer shape: (28709, 7)
x_test_fer shape: (3589, 48, 48, 1), y_test_fer shape: (3589, 7)


In [7]:
# Build Sign Language Detection Model
model_sign = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(24, activation='softmax')
])

model_sign.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_sign.fit(X_train_sign, y_train_sign, epochs=10, batch_size=32, validation_data=(X_test_sign, y_test_sign))

# Build Emotion Detection FER Model
model_fer = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(48, 48, 1)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')
])

C:\Users\Ankush\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 78s 82ms/step - accuracy: 0.7908 - loss: 0.7367 - val_accuracy: 0.7054 - val_loss: 0.9541
Epoch 2/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 67s 79ms/step - accuracy: 0.9841 - loss: 0.0489 - val_accuracy: 0.9480 - val_loss: 0.2074
Epoch 3/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 70s 82ms/step - accuracy: 0.9999 - loss: 7.5714e-04 - val_accuracy: 0.9527 - val_loss: 0.2255
Epoch 4/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 69s 80ms/step - accuracy: 1.0000 - loss: 2.7923e-04 - val_accuracy: 0.9562 - val_loss: 0.2011
Epoch 5/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 70s 81ms/step - accuracy: 0.9765 - loss: 0.0866 - val_accuracy: 0.7068 - val_loss: 1.4037
Epoch 6/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 64s 75ms/step - accuracy: 0.9953 - loss: 0.0158 - val_accuracy: 0.9374 - val_loss: 0.1888
Epoch 7/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 69s 80ms/step - accuracy: 1.0000 - loss: 2.7333e-04 - val_accuracy: 0.9547 - val_loss: 0.1425
Epoch 8/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 69s 80ms/step - accuracy: 0.9947 - los

In [17]:
model_fer.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

# Define data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

# Apply data augmentation to the training data
x_train_fer_aug = data_augmentation(x_train_fer)
y_train_fer_aug = y_train_fer

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epochs: 1e-3 * 10**(-epochs/20))
    
model_fer.fit(x_train_fer, y_train_fer, batch_size=128, epochs=50, validation_data=(x_test_fer, y_test_fer), shuffle=True, callbacks=[early_stopping, lr_schedule])

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(np.argmax(y_train_sign, axis=1)),
    y=np.argmax(y_train_sign, axis=1)
)

class_weights_dict = dict(enumerate(class_weights))

model_sign.fit(X_train_sign, y_train_sign, epochs=10, class_weight=class_weights_dict)

model_fer.save_weights(r"D:\MCA project\Extra Projects\fer2013_new.weights.h5")
model_fer.load_weights(r"D:\MCA project\Extra Projects\fer2013_new.weights.h5")

# Evaluate the model
score = model_fer.evaluate(x_test_fer, y_test_fer)
print('%s: %.2f%%' % (model_fer.metrics_names[1], score[1] * 100))

Epoch 1/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 262s 1s/step - accuracy: 0.3780 - loss: 1.6228 - val_accuracy: 0.3068 - val_loss: 1.7830 - learning_rate: 0.0010
Epoch 2/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 162s 716ms/step - accuracy: 0.5248 - loss: 1.2559 - val_accuracy: 0.4781 - val_loss: 1.3669 - learning_rate: 8.9125e-04
Epoch 3/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 78s 348ms/step - accuracy: 0.5829 - loss: 1.1026 - val_accuracy: 0.5673 - val_loss: 1.1573 - learning_rate: 7.9433e-04
Epoch 4/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 81s 358ms/step - accuracy: 0.6378 - loss: 0.9670 - val_accuracy: 0.5135 - val_loss: 1.3783 - learning_rate: 7.0795e-04
Epoch 5/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 85s 376ms/step - accuracy: 0.6919 - loss: 0.8239 - val_accuracy: 0.5620 - val_loss: 1.2215 - learning_rate: 6.3096e-04
Epoch 6/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 84s 374ms/step - accuracy: 0.7484 - loss: 0.6786 - val_accuracy: 0.5743 - val_loss: 1.2417 - learning_rate: 5.6234e-04
Epoch 7/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 84s 371ms/s

In [35]:

sign_labels = [chr(i) for i in range(65, 65+24)]
emotion_labels = ['angry', 'disgust', 'fearful', 'happy', 'sad', 'surprise', 'netural']


# Update the testing output
img_index = np.random.randint(x_test_fer.shape[0])
sample_img = x_test_fer[img_index]
pred_cls_fer = np.argmax(model_fer.predict(sample_img.reshape(1, 48, 48, 1), verbose=0))
true_cls_fer = np.argmax(y_test_fer[img_index])
print(f'> testing image index: {img_index}\n> true emotion: {emotion_labels[true_cls_fer]}\n> predicted emotion: {emotion_labels[pred_cls_fer]}')


> testing image index: 754
> true emotion: netural
> predicted emotion: netural


In [41]:
# Preprocessing functions for frames
def preprocess_frame(frame, size):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, size) / 255.0
    return np.expand_dims(frame, axis=-1)

def get_prediction(frame, model, size):
    preprocessed_frame = preprocess_frame(frame, size)
    prediction = model.predict(np.expand_dims(preprocessed_frame, axis=0), verbose=0)
    return np.argmax(prediction)






In [43]:
# Capture Video
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)

if not cap.isOpened():
    print("Error: Could not open video capture.")
    exit()


# Update real-time video display
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    sign_prediction = get_prediction(frame, model_sign, (28, 28))
    emotion_prediction = get_prediction(frame, model_fer, (48, 48))

    cv2.rectangle(frame, (10, 10), (250, 100), (0, 0, 0), -1)
    cv2.putText(frame, f"Sign: {sign_labels[sign_prediction]}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.putText(frame, f"Emotion: {emotion_labels[emotion_prediction]}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow('Sign Language and Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
